<a href="https://colab.research.google.com/github/pranjalkamboj/Fine-tuning-Lora-Qlora/blob/main/Qlora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q accelerate peft bitsandbytes transformers trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer


In [3]:
!rm -rf /root/.cache/huggingface/datasets

In [4]:
dataset = load_dataset("tatsu-lab/alpaca", )
dataset = dataset['train'].shuffle(seed=42).select(range(1000))
print(dataset[0])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.47k [00:00<?, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

{'instruction': 'What would be the best type of exercise for a person who has arthritis?', 'input': '', 'output': 'For someone with arthritis, the best type of exercise would be low-impact activities like yoga, swimming, or walking. These exercises provide the benefits of exercise without exacerbating the symptoms of arthritis.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat would be the best type of exercise for a person who has arthritis?\n\n### Response:\nFor someone with arthritis, the best type of exercise would be low-impact activities like yoga, swimming, or walking. These exercises provide the benefits of exercise without exacerbating the symptoms of arthritis.'}


In [5]:
def convert_to_llama2_format(example):
    system_prompt = "You are a helpful assistant."

    instruction = example['instruction'].strip()
    input_text = example['input'].strip()
    output_text = example['output'].strip()

    if input_text == "":
        full_instruction = instruction
    else:
        full_instruction = f"{instruction}\n{input_text}"

    formatted_prompt = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{full_instruction} [/INST] {output_text}</s>"

    return {"text": formatted_prompt}

# Apply the formatting
formatted_dataset = dataset.map(convert_to_llama2_format)
print(formatted_dataset[0]['text'])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

<s>[INST] <<SYS>>
You are a helpful assistant.
<</SYS>>

What would be the best type of exercise for a person who has arthritis? [/INST] For someone with arthritis, the best type of exercise would be low-impact activities like yoga, swimming, or walking. These exercises provide the benefits of exercise without exacerbating the symptoms of arthritis.</s>


In [6]:
## LOra Congig paarameters
lora_dropout=0.1
lora_alpha=16
lora_r=64

In [7]:
## bitsandbytes parameters
use_4bits=True
bnb_4bit_compute_dtype="float16"
bnb_4bit_use_double_quant=True
bnb_4bit_quant_type="nf4"
use_nested_quant=False


In [8]:
##Training parameters
output_dir="./results"
num_train_epochs=1
per_device_train_batch_size=4
fp16=False
bf16=False
per_device_eval_batch_size=4
gradient_accumulation_steps=1
gradient_checkpointing=True
max_grad_norm=0.3
learning_rate=2e-4
weight_decay=0.001
optim="paged_adamw_32bit"
lr_scheduler_type="cosine"
warmup_ratio=0.03
group_by_length=True
save_steps=0
logging_steps=25
max_steps= -1
group_by_length=True
save_steps=0
logging_steps=25


In [9]:
#supervised fine tuning parameters
max_seq_length=None
packing=False
device_map={"":0}

In [10]:
##model names
model_name = "NousResearch/Llama-2-7b-chat-hf"
dataset_name="formatted_dataset"
new_model="llama-2-7b-chat-alpaca"

In [11]:
## Quantizing(bnb config)
compute_dtype=getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bits,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,

)



In [16]:
## Checking our GPU computation power
if compute_dtype == torch.float16 and use_4bits:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
    else:
      print("false")


false


In [15]:
## loading and quantizing the model
model= AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
    quantization_config=bnb_config,

)
model.config.pretraining_tp=1
model.config.use_cache = False



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
## Loading the tokenizer sutiable with our base model
tokenizer=AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
tokenizer.pad_token= tokenizer.eos_token
tokenizer.padding_side="right"


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [18]:
##Loading the LORA layer configration
peft_config=LoraConfig(
    lora_dropout=lora_dropout,
    lora_alpha=lora_alpha,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)